<div id="singlestore-header" style="display: flex; background-color: rgba(209, 153, 255, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/vector-circle.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">Author: Anna Semjen</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">The one billion row challenge with Singlestore</h1>
    </div>
</div>

# The challenge
<div style="width: 60%">
Although the one billion row challenge comes with a Java library, and some detailed instructions on how to generate the file, we will use a pre-loaded file here from S3 to save you from the dirty details of Java compilers and mvnw cleans. 
</span>

# Table of Contents
* [Creating the database for testing](#initiating)
* [Table creation](#quickstart)
* [Pipeline creation](#pipeline)
* [Querying the min average and the max](#last)

# Creating the database for testing <a class="anchor" id="initiating"></a>

In [5]:
%%sql
DROP DATABASE IF EXISTS onebrc;
CREATE DATABASE onebrc;
use onebrc;

++
||
++
++

# Table creation <a class="anchor" id="quickstart"></a>

In [7]:
%%sql
drop table if exists cities; 
CREATE TABLE cities(
city text(20) CHARACTER SET utf8 COLLATE utf8_general_ci,
temperature text(6) CHARACTER SET utf8 COLLATE utf8_general_ci,
SHARD key(city),
SORT KEY(temperature));

++
||
++
++

# Pipeline creation <a class="anchor" id="pipeline"></a>

In [8]:
%%sql
CREATE or replace PIPELINE onebrc AS
LOAD DATA S3 's2db-demos-pub-bucket/billion-row-challenge/measurements.txt'
INTO TABLE cities
fields terminated by ";"
    ;


++
||
++
++

In [10]:
%sql START PIPELINE onebrc;

++
||
++
++

# Collecting statistics on the batch <a class="anchor" id="quickstart"></a>

In [95]:
%%sql
select batch_time/60 as min ,*
 from information_schema.PIPELINES_BATCHES where pipeline_name = 'onebrc' and database_name = 'onebrc' 
    and batch_state in ('In Progress', 'Succeeded')
         ;

min,DATABASE_NAME,PIPELINE_NAME,BATCH_ID,BATCH_STATE,BATCH_ROWS_WRITTEN,BATCH_TIME,BATCH_START_UNIX_TIMESTAMP,BATCH_PARTITION_STATE,BATCH_PARTITION_PARSED_ROWS,BATCH_SOURCE_PARTITION_ID,BATCH_EARLIEST_OFFSET,BATCH_LATEST_OFFSET,BATCH_PARTITION_TIME,BATCH_PARTITION_EXTRACTED_BYTES,BATCH_PARTITION_TRANSFORMED_BYTES,BATCH_PARTITION_EXTRACTOR_WAIT_TIME,BATCH_PARTITION_TRANSFORM_WAIT_TIME,HOST,PORT,PARTITION
6.153865533333333,onebrc,onebrc,578771,Succeeded,1000000000,369.231932,1706697029.846055,Succeeded,1000000000,billion-row-challenge/measurements.txt,0,1,300.190435,13795372113,None,12.9652,0.0,node-7957c397-aa8a-4dff-b0ef-781604bce52c-leaf-ag1-1.svc-7957c397-aa8a-4dff-a565-b2f965de22d9,3306,11


# Querying the min the average the max  <a class="anchor" id="last"></a>

In [78]:
%%sql
select GROUP_CONCAT( '{', city, '=', mint, '/',avgt, '/', maxt,  '}'
 order by city)
from (
       Select city, 
              min(temperature :> float(4,1)) as mint, 
              max(temperature :> float(4,1)) as maxt, 
              avg(temperature :> float(4,1)) as avgt from cities
       Group by city);

"GROUP_CONCAT( '{', city, '=', mint, '/',avgt, '/', maxt, '}' order by city)"


<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>